*Load packages and data:*

In [1]:
%run stdPackages.ipynb
read = {'variables': ['Fundamentals', 'Load', 'Generators_Other','TL','Regulation'], 
        'variable2D': ['Generators_FuelMix','HourlyVariation'],
        'scalars': ['Scalars'],
        'maps': ['Generators_Categories', 'Load_Categories']}
db = dbFromWB(os.path.join(d['data'],'E3.xlsx'), read)
readSets(db)

# Exercise 3: The value of transmission lines and the cost of intermittency

The main objectives of the third exercise are:
1. Investigating the economic value of trade in electricity;
2. explore its role in mitigating the costs of intermittency;
3. analyze policies aimed at reducing CO2 emissions.

The exercise builds on the ```mBasicTrade``` model class; you can find helpful information on this in the following:
* The note "*Models for Energy Economics*" describes the model in section 3.
* The notebook [M_mBasicTrade](M_mBasicTrade.ipynb) includes some description of the model class.
* The notebook [E3_SolutionGuide](E3_SolutionGuide.ipynb) provides a short solution to the problems posted in the current notebook.

## The setup:

We consider a ```mBasicTrade``` model with two geographic markets $g1,g2$ and four hours $H=4$.
* The $g1$ market is taken from [Exercise 2](E2_Problem.ipynb): There are four plants that contribute to satisfy a "daily" load of $400GJ$. Three plants combine fuels: Coal, natural gas, and biomass. The fourth plant produces wind power.
* The $g2$ market is very simple: There is daily load of $400GJ$ as well, but only two plants (id5-id6). One that relies on natural gas (id5) and a solar (photovoltaics, PV, id6) plant.
* The two markets are connected via a transmission line with hourly capacity of $25GJ$ exports (```lineCapacity```). When exporting, there is a loss of 5\% of the transported electricity (```lineLoss```). Thus, the maximum imports are $25*0.95$ GJ.

As in exercise 2, the day consists of four short run states. They are identical in the two areas.  

The plant ```id```s are tied to a specific geographic area in the mapping ```id2g``` stored in the database:

In [2]:
db['id2g']

MultiIndex([('id1', 'g1'),
            ('id2', 'g1'),
            ('id3', 'g1'),
            ('id4', 'g1'),
            ('id5', 'g2'),
            ('id6', 'g2')],
           names=['id', 'g'])

Some descriptive figures are plotted here for area $g2$ (similar to those used in exercise 2 as well)

In [3]:
IFrame("Misc/Figs/mBasicTrade_hvt_g2.pdf", width=600, height=300)

In [4]:
IFrame("Misc/Figs/mBasicTrade_MeritOrder_g2.pdf", width=600, height=300)

### Problem 1: Predicted trade flows

*Hypothesize on the trade patterns that emerges this model: In what hours of the day do you expect $g1, g2$ to benefit from trade in electricity? Confirm your hypothesis by solving the model and inspecting the solution variable ```Transmission[g,g_alias,h]``` which is stores the solution $x_{g,g',h}$ measuring the export from $g$ to $g'$ in hour $h$.*

### Problem 2: Gains of trade

Solve the model with varying transmission capacity (variable ```lineCapacity```) and inspect/explain:
1. *Marginal system costs in the two areas. Do you expect them to increase/decrease when you increase transmission capacities?*
2. *Congestion rents.* 
3. *Marginal economic value of the 6 plants.*

*Hint 1: All three variables you are asked to inspect are computed automatically and can be accessed as ```marginalSystemCosts```, ```congestionRent```, and ```marginalEconomicValue``` respectively.*

*Hint 2: The congestion rent in an hour $h$ defined as*
$$\begin{align}
    \text{Congestion rent}_h = \sum_{l} x_{l,h} \cdot (p_{g',h} - p_{g,h}) \geq 0.
\end{align}$$
*where $x_{l,h}$ is the export from $g$ to $g'$ and $p_g, p_{g'}$ are the prices in the two markets. So, congestion rents **measure the increase in economic value that comes from exporting $x_{l,h}$ to a high-price market $(p_{g'})$ instead of consuming it locally $(p_{g})$.***

*In our model these variables are defined as:*
* *```Transmission[g,g_alias,h]```$=x_{g,g',h}$,*
* *```marginalSystemCosts[g,h]``` $=p_{g,h}$.*
* *```marginalEconomicValue[id]``` measures the marginal economic value*

### Problem 3: Coordinated/uncoordinated regulation

Another topic that arises with multiple markets/zones/areas (```g```) is that of coordinated/uncoordinated regulation. This topic is a classic in climate economics and will covered in more detail in other courses (e.g. *"Economics of the Environment and Climate Change"* at UCPH). To look into this issue, start by initializing a model with emission constraints as follows:

In [5]:
m = mBasicTrade.mEmissionCap(db)

This model class allows us to impose emission constraints on the two geographic areas separately (uncoordinated) or as one common (coordinated) emissions cap by setting:

In [6]:
m.commonCap = True # default option, coordinated regulation.
m.commonCap = False # Use this for uncoordinated regulation.

Revert the transmission capacity to its original setting of 25 GJ:

In [7]:
m.db['lineCapacity'] = 25

*Simulate the effect of lowering the emission cap from 50 to 20. Compare the effect of doing so coordinated (total cap of 50), and uncoordinated (cap of 25/25).*

*Hint 1: The shock requires the following grid specification*

In [8]:
loop = pd.Index(range(11), name = 'loop') # number of gridpoints.
v0 = pd.Series(25, index = m.db['g']) # start with cap on 25/25.
vT = pd.Series(10, index = m.db['g']) # end with cap 10/10.
grid = addGrid(v0,vT,loop,'CO2Cap') # grid

*Hint 2: The variable ```Welfare``` measures the objective function.*